In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action = 'ignore')

# 데이터 불러오기

In [2]:
data = pd.read_csv('data_preprocessing_2.csv')
data.head()

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500


In [63]:
submission = pd.read_csv('data/submission.csv', index_col=0)
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,0
1,202004,강원,골프장 운영업,0
2,202004,강원,과실 및 채소 소매업,0
3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0


# 2020, 2019 df 쪼개기

In [3]:
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]

data_2019.shape, data_2020.shape

((2169935, 11), (521750, 11))

In [4]:
data_2019.head()

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500


In [5]:
data_2020.head()

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
2169935,2020,1,강원,강릉시,건강보조식품 소매업,1,0,30,3,3,1903450
2169936,2020,1,강원,강릉시,건강보조식품 소매업,1,0,40,12,13,1845900
2169937,2020,1,강원,강릉시,건강보조식품 소매업,1,0,50,39,38,6132260
2169938,2020,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,18,3162700
2169939,2020,1,강원,강릉시,건강보조식품 소매업,1,0,70,7,9,554000


# 2019년 3월 4월 비율 만들기

In [6]:
## 
data_2019_sum = data_2019.groupby(['month', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum()[['AMT']].reset_index()
data_2019_march = data_2019_sum[data_2019_sum['month'] == 3]
data_2019_april = data_2019_sum[data_2019_sum['month'] == 4]
# data_2019_march['CARD_CCG_NM'].unique().shape
# data_2019_march['STD_CLSS_NM'].unique().shape

data_2019_march.shape, data_2019_april.shape

((630, 4), (633, 4))

In [12]:
data_2019_march['AMT_march'] = data_2019_march['AMT']
data_2019_april['AMT_april'] = data_2019_april['AMT']

In [13]:
data_2019_march = data_2019_march.drop(['month', 'AMT'], axis = 1)
data_2019_april = data_2019_april.drop(['month', 'AMT'], axis = 1)

In [15]:
data_2019_mar_apr = pd.merge(data_2019_march ,data_2019_april , how = 'right')
# data_2019_2020['2019/2020'] = data_2019_2020 / data_2019_2020
data_2019_mar_apr['apr/mar'] = data_2019_mar_apr['AMT_march'] / data_2019_mar_apr['AMT_april']
data_2019_mar_apr

,CARD_SIDO_NM,STD_CLSS_NM,AMT_march,AMT_april,apr/mar
0,강원,건강보조식품 소매업,1.094390e+08,106548473,1.027129
1,강원,골프장 운영업,2.760497e+09,3964893280,0.696235
2,강원,과실 및 채소 소매업,9.392280e+08,827683360,1.134767
3,강원,관광 민예품 및 선물용품 소매업,2.502319e+07,17905610,1.397506
4,강원,그외 기타 스포츠시설 운영업,6.817720e+06,5904100,1.154743
...,...,...,...,...,...
628,강원,면세점,NaN,113730,NaN
629,울산,면세점,NaN,791000,NaN
630,제주,기타 수상오락 서비스업,NaN,59000,NaN
631,충남,택시 운송업,NaN,64120,NaN


In [22]:
rate_march_apr = data_2019_mar_apr.groupby('STD_CLSS_NM').mean()[['apr/mar']].reset_index()
rate_march_apr

,STD_CLSS_NM,apr/mar
0,건강보조식품 소매업,1.035321
1,골프장 운영업,0.849998
2,과실 및 채소 소매업,1.021717
3,관광 민예품 및 선물용품 소매업,1.110893
4,그외 기타 분류안된 오락관련 서비스업,1.795163
5,그외 기타 스포츠시설 운영업,1.610999
6,그외 기타 종합 소매업,1.059601
7,기타 대형 종합 소매업,1.044640
8,기타 수상오락 서비스업,1.431247
9,기타 외국식 음식점업,1.060621


In [26]:
result_2020 =  data_2020_march.merge(rate_march_apr, how = 'left')
result_2020['result'] = result_2020['AMT'] * result_2020['apr/mar']
result_2020['REG_YYMM'] = 202004

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT,apr/mar,result
0,3,강원,건강보조식품 소매업,96059012,1.035321,9.945196e+07
1,3,강원,골프장 운영업,2915797995,0.849998,2.478423e+09
2,3,강원,과실 및 채소 소매업,994816943,1.021717,1.016422e+09
3,3,강원,관광 민예품 및 선물용품 소매업,13317300,1.110893,1.479410e+07
4,3,강원,그외 기타 스포츠시설 운영업,2075000,1.610999,3.342822e+06
...,...,...,...,...,...,...
604,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299,1.005557,1.322554e+09
605,3,충북,한식 음식점업,16152482704,1.053174,1.701137e+10
606,3,충북,호텔업,15248550,1.090038,1.662150e+07
607,3,충북,화장품 및 방향제 소매업,428881434,1.034000,4.434634e+08


In [35]:
result_2020.drop(['month', 'AMT', 'apr/mar'], axis = 1, inplace = True)

In [36]:
result_2020

,CARD_SIDO_NM,STD_CLSS_NM,result,REG_YYMM
0,강원,건강보조식품 소매업,9.945196e+07,202004
1,강원,골프장 운영업,2.478423e+09,202004
2,강원,과실 및 채소 소매업,1.016422e+09,202004
3,강원,관광 민예품 및 선물용품 소매업,1.479410e+07,202004
4,강원,그외 기타 스포츠시설 운영업,3.342822e+06,202004
...,...,...,...,...
604,충북,피자 햄버거 샌드위치 및 유사 음식점업,1.322554e+09,202004
605,충북,한식 음식점업,1.701137e+10,202004
606,충북,호텔업,1.662150e+07,202004
607,충북,화장품 및 방향제 소매업,4.434634e+08,202004


In [67]:
submission_1 = submission.drop(['AMT'], axis=1)
submission_1 = submission_1.merge(result_2020, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how = 'left')
submission_1 = submission_1.fillna(0)
submission_1.index.name = 'id'
submission_1.to_csv('submission_1.csv', encoding='utf-8-sig')
submission_1.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,result
id,,,,
0,202004,강원,건강보조식품 소매업,9.945196e+07
1,202004,강원,골프장 운영업,2.478423e+09
2,202004,강원,과실 및 채소 소매업,1.016422e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.479410e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [73]:
submission_1['AMT'] = submission_1['result']
submission_1 = submission_1.drop('result', axis = 1)
submission_1.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.945196e+07
1,202004,강원,골프장 운영업,2.478423e+09
2,202004,강원,과실 및 채소 소매업,1.016422e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.479410e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00


In [74]:
submission_1.to_csv('submission_1.csv', encoding='utf-8-sig')

In [71]:
pd.read_csv('submission.csv', index_col=0)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,result
id,,,,
0,202004,강원,건강보조식품 소매업,9.945196e+07
1,202004,강원,골프장 운영업,2.478423e+09
2,202004,강원,과실 및 채소 소매업,1.016422e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.479410e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0.000000e+00
1390,202007,충북,한식 음식점업,0.000000e+00
1391,202007,충북,호텔업,0.000000e+00


In [70]:
pd.read_csv('submission_1.csv', index_col=0)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,result
id,,,,
0,202004,강원,건강보조식품 소매업,9.945196e+07
1,202004,강원,골프장 운영업,2.478423e+09
2,202004,강원,과실 및 채소 소매업,1.016422e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.479410e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0.000000e+00
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0.000000e+00
1390,202007,충북,한식 음식점업,0.000000e+00
1391,202007,충북,호텔업,0.000000e+00


In [59]:
sub1.to_csv('submission.csv', encoding='utf-8-sig')

In [ ]:
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

In [56]:
sub1.to_csv('sub1.csv', encoding = 'utf-8', index = False)

In [49]:
sub1.fillna(0)
sub1.isnull().sum()

id                0
REG_YYMM          0
CARD_SIDO_NM      0
STD_CLSS_NM       0
AMT             785
dtype: int64

In [43]:
data_2020_march

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT
1251,3,강원,건강보조식품 소매업,96059012
1252,3,강원,골프장 운영업,2915797995
1253,3,강원,과실 및 채소 소매업,994816943
1254,3,강원,관광 민예품 및 선물용품 소매업,13317300
1255,3,강원,그외 기타 스포츠시설 운영업,2075000
...,...,...,...,...
1855,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
1856,3,충북,한식 음식점업,16152482704
1857,3,충북,호텔업,15248550
1858,3,충북,화장품 및 방향제 소매업,428881434


In [25]:
result_2020

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT,apr/mar
0,3,강원,건강보조식품 소매업,96059012,1.035321
1,3,강원,골프장 운영업,2915797995,0.849998
2,3,강원,과실 및 채소 소매업,994816943,1.021717
3,3,강원,관광 민예품 및 선물용품 소매업,13317300,1.110893
4,3,강원,그외 기타 스포츠시설 운영업,2075000,1.610999
...,...,...,...,...,...
604,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299,1.005557
605,3,충북,한식 음식점업,16152482704,1.053174
606,3,충북,호텔업,15248550,1.090038
607,3,충북,화장품 및 방향제 소매업,428881434,1.034000


In [133]:
result_april = []

for i in data_2020_march.values:
    rate = rate_march_apr.loc[i[2]].values
    result_april.append(rate*i[3])

In [147]:
rate_march_apr.loc[i[2]]

apr/mar    1.148639
Name: 휴양콘도 운영업, dtype: float64

In [117]:
data_2020_march['result'] = result_april

ValueError: Length of values does not match length of index

In [114]:
data_2020_march

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT,result
1192,3,강원,건강보조식품 소매업,96059012,[98513440.93099493]
1193,3,강원,골프장 운영업,2915797995,[2459422801.559337]
1194,3,강원,과실 및 채소 소매업,994816943,[1014503858.5371922]
1195,3,강원,관광 민예품 및 선물용품 소매업,13317300,[14020144.558248919]
1196,3,강원,그외 기타 스포츠시설 운영업,2075000,[3342822.095073291]
...,...,...,...,...,...
1768,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299,[1326154723.4984238]
1769,3,충북,한식 음식점업,16152482704,[17038576597.24758]
1770,3,충북,호텔업,15248550,[16621504.585944735]
1771,3,충북,화장품 및 방향제 소매업,428881434,[442053742.53346634]


In [43]:
data_2019_march['AMT_2019'] = data_2019_march['AMT']
data_2019_march.drop('AMT', axis = 1, inplace = True)
data_2019_march

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT_2019
1199,3,강원,건강보조식품 소매업,109439025
1200,3,강원,골프장 운영업,2760497139
1201,3,강원,과실 및 채소 소매업,939227950
1202,3,강원,관광 민예품 및 선물용품 소매업,25023190
1203,3,강원,그외 기타 스포츠시설 운영업,6817720
...,...,...,...,...
1795,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1441131811
1796,3,충북,한식 음식점업,22952482750
1797,3,충북,호텔업,36030010
1798,3,충북,화장품 및 방향제 소매업,729328422


In [7]:
data_2020_sum = data_2020.groupby(['month', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum()[['AMT']].reset_index()
data_2020_march = data_2020_sum[data_2020_sum['month'] == 3]
# data_2020_march['CARD_CCG_NM'].unique().shape
# data_2020_march['STD_CLSS_NM'].unique().shape

In [44]:
data_2020_march['AMT_2020'] = data_2020_march['AMT']
data_2020_march.drop('AMT', axis = 1, inplace = True)
data_2020_march

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT_2020
1192,3,강원,건강보조식품 소매업,96059012
1193,3,강원,골프장 운영업,2915797995
1194,3,강원,과실 및 채소 소매업,994816943
1195,3,강원,관광 민예품 및 선물용품 소매업,13317300
1196,3,강원,그외 기타 스포츠시설 운영업,2075000
...,...,...,...,...
1768,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1315245299
1769,3,충북,한식 음식점업,16152482704
1770,3,충북,호텔업,15248550
1771,3,충북,화장품 및 방향제 소매업,428881434


In [46]:
data_2019_2020 = pd.merge(data_2019_march ,data_2020_march , how = 'left')
data_2019_2020['2019/2020'] = data_2019_2020 / data_2019_2020

,month,CARD_SIDO_NM,STD_CLSS_NM,AMT_2019,AMT_2020
0,3,강원,건강보조식품 소매업,109439025,9.605901e+07
1,3,강원,골프장 운영업,2760497139,2.915798e+09
2,3,강원,과실 및 채소 소매업,939227950,9.948169e+08
3,3,강원,관광 민예품 및 선물용품 소매업,25023190,1.331730e+07
4,3,강원,그외 기타 스포츠시설 운영업,6817720,2.075000e+06
...,...,...,...,...,...
596,3,충북,피자 햄버거 샌드위치 및 유사 음식점업,1441131811,1.315245e+09
597,3,충북,한식 음식점업,22952482750,1.615248e+10
598,3,충북,호텔업,36030010,1.524855e+07
599,3,충북,화장품 및 방향제 소매업,729328422,4.288814e+08


In [ ]:
# 20년 3월 + (19년4월 - 19년 3월) * (20년 3월 /19년 3월)

In [ ]:
weight = {}
sido_list = data_2019['CARD_SIDO_NM'].unique()
std_list = data_2019['STD_CLSS_NM'].unique()

for sido in sido_list:
    for std in std_list:
        df = data_2019[(data_2019['CARD_SIDO_NM'] == sido) & (data_2019['STD_CLSS_NM'] == std)]
        

In [ ]:
sido = data_2019['CARD_SIDO_NM'].unique()[0]
s

In [8]:
data_2020['month'].unique(), data_2019['month'].unique()

(array([1, 2, 3], dtype=int64), array([1, 2, 3], dtype=int64))

In [10]:
sido = data['CARD_SIDO_NM'].unique()[0]
std = data['STD_CLSS_NM'].unique()[0]

sido, std

('강원', '건강보조식품 소매업')

In [13]:
test_data_2019 = data_2019[(data_2019['CARD_SIDO_NM'] == sido) & (data_2019['STD_CLSS_NM'] == std)]
test_data_2019

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500
...,...,...,...,...,...,...,...,...,...,...,...
360857,2019,3,강원,횡성군,건강보조식품 소매업,1,0,50,5,4,958000
360858,2019,3,강원,횡성군,건강보조식품 소매업,1,0,60,5,5,520000
360859,2019,3,강원,횡성군,건강보조식품 소매업,2,0,40,3,3,520000
360860,2019,3,강원,횡성군,건강보조식품 소매업,2,0,50,12,13,1269225


In [14]:
test_data_2020 = data_2020[(data_2020['CARD_SIDO_NM'] == sido) & (data_2020['STD_CLSS_NM'] == std)]
test_data_2020

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
2163110,2020,1,강원,강릉시,건강보조식품 소매업,1,0,30,3,3,1903450
2163111,2020,1,강원,강릉시,건강보조식품 소매업,1,0,40,12,13,1845900
2163112,2020,1,강원,강릉시,건강보조식품 소매업,1,0,50,39,38,6132260
2163113,2020,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,18,3162700
2163114,2020,1,강원,강릉시,건강보조식품 소매업,1,0,70,7,9,554000
...,...,...,...,...,...,...,...,...,...,...,...
2522944,2020,3,강원,홍천군,건강보조식품 소매업,2,0,60,4,4,90000
2522945,2020,3,강원,홍천군,건강보조식품 소매업,2,0,70,3,2,375000
2523896,2020,3,강원,횡성군,건강보조식품 소매업,1,0,50,13,13,1755215
2523897,2020,3,강원,횡성군,건강보조식품 소매업,2,0,50,3,3,550000


In [17]:
test_data_2019[test_data_2019['month'] == 1][:50]

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500
5,2019,1,강원,강릉시,건강보조식품 소매업,1,0,70,7,7,660000
6,2019,1,강원,강릉시,건강보조식품 소매업,2,0,30,6,6,818700
7,2019,1,강원,강릉시,건강보조식품 소매업,2,0,40,7,7,627500
8,2019,1,강원,강릉시,건강보조식품 소매업,2,0,50,21,20,3573300
9,2019,1,강원,강릉시,건강보조식품 소매업,2,0,60,33,40,7194600


In [28]:
test_data_2019[test_data_2019['CARD_CCG_NM'] == '강릉시']
# plt.plot(x = test_data_2019['month'], y = test_data_2019['AMT'], hue = test_data_2019['AGE'])

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500
5,2019,1,강원,강릉시,건강보조식품 소매업,1,0,70,7,7,660000
6,2019,1,강원,강릉시,건강보조식품 소매업,2,0,30,6,6,818700
7,2019,1,강원,강릉시,건강보조식품 소매업,2,0,40,7,7,627500
8,2019,1,강원,강릉시,건강보조식품 소매업,2,0,50,21,20,3573300
9,2019,1,강원,강릉시,건강보조식품 소매업,2,0,60,33,40,7194600


In [76]:
data

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,0,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,0,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,0,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,0,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,0,60,19,20,1433500
...,...,...,...,...,...,...,...,...,...,...,...
2683213,2020,3,충북,충주시,휴양콘도 운영업,1,0,30,3,4,43300
2683214,2020,3,충북,충주시,휴양콘도 운영업,1,0,40,3,3,35000
2683215,2020,3,충북,충주시,휴양콘도 운영업,1,0,50,4,6,188000
2683216,2020,3,충북,충주시,휴양콘도 운영업,1,0,60,3,3,194000


In [79]:
sido_list = data['CARD_SIDO_NM'].unique()
std_list = data['STD_CLSS_NM'].unique()

for sido in sido_list:
    for std in std_list:
        df = data[(data['CARD_SIDO_NM'] == sido) & (data['STD_CLSS_NM'] == std)]
        df1 = pd.get_dummies(df, columns = ['CARD_CCG_NM', 'SEX', 'region_diff', 'AGE'])
        df1.head()

KeyboardInterrupt: 

In [87]:
sido = data['CARD_SIDO_NM'].unique()[-1]
std = data['STD_CLSS_NM'].unique()[-1]
data[(data['CARD_SIDO_NM'] == sido) & (data['STD_CLSS_NM'] == std)]

(0, 11)